In [39]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load the dataset
df = pd.read_csv(r"C:\Users\yerra\Desktop\BMW Project\dataset.csv")

# Define categories and assign models to categories
categories = {
    'Electric Vehicles': [
        'BMW i4 eDrive35', 'BMW i4 eDrive40', 'BMW i4 xDrive40', 'BMW i4 M50 xDrive',
        'BMW i7 eDrive50', 'BMW i7 xDrive60', 'BMW i7 M70 xDrive', 'BMW iX xDrive40',
        'BMW iX xDrive50', 'BMW iX M60', 'BMW iX1 eDrive20', 'BMW iX1 xDrive30',
        'BMW iX2 eDrive20', 'BMW iX2 xDrive30', 'BMW iX3', 'MINI Cooper E', 
        'MINI Cooper SE', 'MINI Aceman E', 'MINI Aceman SE', 'MINI Cooper C',
        'MINI Cooper S', 'Rolls-Royce Spectre'
    ],
    'SUVs': [
        'BMW X3 30e xDrive', 'BMW X2 sDrive18d', 'BMW X2 sDrive20d', 'BMW X2 xDrive20d', 
        'BMW X2 sDrive20i', 'BMW X2 M35i xDrive', 'BMW X3 20d xDrive', 'BMW X3 20 xDrive', 
        'BMW X3 M50 xDrive', 'Rolls-Royce Black Badge Cullinan Series II', 
        'Rolls-Royce Cullinan Series II'
    ]
}

# Assign categories
df['Category'] = df['Model'].apply(lambda model: next((cat for cat, models in categories.items() if model in models), 'Other'))

# Initialize the Dash app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Fuel Consumption (l/100km)', 'value': 'Fuel Consumption (l/100km)'},
            {'label': 'Energy Consumption (kWh/100km)', 'value': 'Energy Consumption (kWh/100km)'},
            {'label': 'CO2 Emissions (g/km)', 'value': 'CO2 Emissions (g/km)'}
        ],
        value='Fuel Consumption (l/100km)',
        style={'width': '50%', 'margin-bottom': '10px'}
    ),
    html.Div([
        dcc.Graph(id='ev-chart', style={'width': '48%', 'display': 'inline-block'}),
        dcc.Graph(id='suv-chart', style={'width': '48%', 'display': 'inline-block'})
    ])
])

# Callback to update both charts
@app.callback(
    [Output('ev-chart', 'figure'), Output('suv-chart', 'figure')],
    [Input('metric-dropdown', 'value')]
)
def update_charts(metric):
    # Filter data for Electric Vehicles
    ev_df = df[df['Category'] == 'Electric Vehicles']
    ev_fig = px.line(ev_df, x='Model', y=metric, title='Electric Vehicles', markers=True)
    ev_fig.update_traces(line=dict(color='blue', width=2))
    ev_fig.update_layout(
        xaxis_title='Model',
        yaxis_title=metric,
        xaxis_tickangle=-45,
        showlegend=False,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        yaxis=dict(showgrid=True, gridcolor='lightgray')
    )

    # Filter data for SUVs
    suv_df = df[df['Category'] == 'SUVs']
    suv_fig = px.line(suv_df, x='Model', y=metric, title='SUVs', markers=True)
    suv_fig.update_traces(line=dict(color='orange', width=2))
    suv_fig.update_layout(
        xaxis_title='Model',
        yaxis_title=metric,
        xaxis_tickangle=-45,
        showlegend=False,
        plot_bgcolor='rgba(0, 0, 0, 0)',
        yaxis=dict(showgrid=True, gridcolor='lightgray')
    )

    return ev_fig, suv_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
